# Data Understanding

In dieser Phase beschreibt der Autor die zuerst die Herkunft und Beschaffungsweise der Daten. Daraufhin wird geprüft, ob diese den in der Zielsetzung formulierten Anforderungen gerecht werden.

## 1. Datenbeschaffung

### 1.1 Grundlegende Informationen zum Datensatz

<b>Datenbereitsteller: </b>Medizinische Medien Informations GmbH, Am Forsthaus Gravenbruch 7, 63263 Neu-Isenburg<br>
<b>Datensatzbezeichnung: </b>MMI Pharmaindex - Rohdaten Release R3<br>
<b>Version: </b>1.25<br>
<b>Datum: </b>15.03.2019<br>

<b>Struktur:</b><br>
<code>├── MMI_RohdatenR3
│   ├── doc
│   └── mmiPharmindexR3_20190501MAIN.zip
├── MMI_RohdatenR3_Fachinformation
│   ├── doc
│   └── mmiPharmindexR3_20190501SPC.zip
├── MMI_RohdatenR3_Interaktionen
│   ├── doc
│   └── mmiPharmindexR3_20190501INTERACTION.zip
└── MMI_RohdatenR3_Medplan
    ├── mmiPharmindexR3_20190501MEDPLAN.zip
    └── mmiPharmindexR3_MEDPLAN_Datenstruktur.pdf</code>

Der Datensatz gliedert sich in der ersten Ebene auf die drei Unterordner MMI_RohdatenR3, -Fachinformationen, -Interaktionen und -Medplan auf. Darin enthalten ist zum einen ein doc-Ordner mit einer PDF, in der der jeweilige Datensatz beschrieben wird, zum anderen eine Zip-Datei in der die jeweiligen Datentabellen als CSV-Datei hinterlegt sind.

Der Datensatz ist in dieser Form u.a. bei der Arzneimitteldatenbank <a href="https://gelbe-liste.de">"Gelbe Liste"</a>im Einsatz.

### 1.2 Selektion der benötigten Dateien

Für die Abbildung des Wechselwirkungschecks und einer generellen Übersicht werden nicht alle Dateien benötigt. Ausschließlich die Angaben zu den einzelnen Arzneimitteln, Herstellern und die Interaktionsdaten sind hierfür von Interesse. Erstere sind in die den Rohdaten_R3 (/mmiPharmindexR3_20190501MAIN) zu finden, die Interaktionsdaten in dem gleichnamigen Unterordner (/mmiPharmindexR3_20190501INTERACTION). Fachinformationen und der Medikationsplan sind nicht von Relevanz und werden daher nicht weiter betrachtet.

Im Detail werden folgenden Tabellen innerhalb der Unterordner benötigt:

<ol>
    <li>MMI_RohdatenR3/mmiPharmindexR3_20190501MAIN</li>
        <ul><li>PRODUCT.CSV</li>
            <li>COMPANY.CSV</li>
            <li>COMPANYADRESS.CSV</li>
            <li>PRODUCT_COMPANY.CSV</li></ul>
    <br/>
    <li>MMI_RohdatenR3_Interaktionen/mmiPharmindexR3_20190501INTERACTION</li>
        <ul><li>INTERACTION.CSV</li>
            <li>INTERACTION_CLASS.CSV</li>
            <li>PRODUCT_INTERACTION_CLASS.CSV</li></ul>
</ol>

### 1.3 Importieren von Python-Paketen

In [82]:
import pandas as pd
import csv

### 1.4 Einlesen der benötigten Dateien

In [113]:
# Funktion zum Einlesen
def csv_to_df(csv):
    df = pd.read_csv(csv, header=0, sep=';', quotechar='"', decimal=',', low_memory=False)
    return df

# Produkte
product = csv_to_df("../data/origin/PRODUCT.csv")
product.name = "Produkte"

# Hersteller
company = csv_to_df("../data/origin/COMPANY.csv")
company.name = "Hersteller"

# Hersteller-Adresse
company_adress = csv_to_df("../data/origin/COMPANYADDRESS.csv")
company_adress.name = "Hersteller-Adresse"

# Schnittstelle Hersteller u. Produkt
product_company = csv_to_df("../data/origin/PRODUCT_COMPANY.csv")
product_company.name = "API-Produkt-Hersteller"

# Interaktionen
interaction = csv_to_df("../data/origin/INTERACTION.csv")
interaction.name = "Interaktionen"

# Interaktions-Klasse
interaction_class = csv_to_df("../data/origin/INTERACTION_CLASS.csv")
interaction_class.name = "Interaktionsklasse"

# Schnittstelle zw. Produkte u. Interaktionen
product_interaction_class = csv_to_df("../data/origin/PRODUCT_INTERACTION_CLASS.csv")
product_interaction_class.name = "API-Produkte-Interaktionen"

### Ausgabe einer ersten Analyse

In [114]:
all_data = [product, company, company_adress, product_company, interaction, 
            interaction_class, product_interaction_class]

# Ausgabe einer ersten Analyse
for df in all_data:
    print(df.name + ':')
    print(str(df.info()) + '\n' +
          '--------------------------------------------------------------------' + '\n')

Produkte:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545159 entries, 0 to 545158
Data columns (total 18 columns):
ID                               545159 non-null int64
NAME                             545159 non-null object
NAME_PLAIN                       545159 non-null object
SORTNAME                         545159 non-null object
ONMARKETDATE                     545159 non-null object
REGISTRATION_NUMBER              0 non-null float64
ACTIVESUBSTANCECOUNT             70414 non-null float64
ORIGINALPRODUCTID                13064 non-null float64
DISPENSINGTYPECATALOGID          545159 non-null int64
DISPENSINGTYPECODE               545159 non-null int64
PRODUCTFOODTYPECATALOGID         530051 non-null float64
PRODUCTFOODTYPECODE              530051 non-null object
PRODUCTDIETETICSTYPECATALOGID    545135 non-null float64
PRODUCTDIETETICSTYPECODE         545135 non-null object
PAO_TIMEUNITCATALOGID            2865 non-null float64
PAO_TIMEUNITCODE                 2865 non-nul

### Bereinigung der Dataframes auf die relevanten Spalten

In [109]:
# Produkte (hier reicht die ID, der Name und als zusätzl. Info das Markteintrittsdatum)
product = product[['ID','NAME','ONMARKETDATE']]

# Hersteller (ID, Name)
company = company[['ID','NAME']]

# Hersteller-Adresse (ID, COMPANYID, ZIP, STREET, STREETNUMBER)
company_adress = company_adress[['ID', 'COMPANYID', 'ZIP', 'CITY', 'STREET', 'STREETNUMBER']]

# Schnittstelle zw. Produkte und Hersteller
product_company = product_company[['PRODUCTID', 'COMPANYID']]

# Interaktionen (ID, Interaktionsklasse1, Interaktionsklasse2, Severity, Likelihood)
interaction = interaction[['INTERACTIONCLASSID1', 'INTERACTIONCLASSID2', 'SEVERITY', 'LIKELIHOOD']]

# ! Die beiden anderen Interaktionstabellen benötigen keine Veränderung !

all_data_final = [product, company, company_adress, product_company, 
                  interaction, interaction_class, product_interaction_class]

### Bereinigung von einzelnen Attributen und Zusammenfügen von Datensätzen

In [110]:
# Anpassen des Datums bei ONMARKETDATE in product --> MySQL arbeitet mit amerikanischem Datumsformat
product['ONMARKETDATE'] = pd.to_datetime(product['ONMARKETDATE'], format='%d.%m.%Y')

# Merge aus den Produkten und Herstellern
prod_comp_mrg = pd.merge(product, product_company, left_on='ID', right_on='PRODUCTID')
print(prod_comp_mrg.head())
print(prod_comp_mrg.info())

   ID                                    NAME ONMARKETDATE  PRODUCTID  \
0  14        Trental® 400 mg, Retardtabletten   1977-09-01         14   
1  21         BETOPTIMA® 5 mg/ml Augentropfen   1985-09-01         21   
2  30    Cyclopentolat Alcon® 1% Augentropfen   1998-04-01         30   
3  32         DEXA-SINE® 1 mg/ml Augentropfen   1998-12-01         32   
4  33  DEXA-SINE® SE 1,315 mg/ml Augentropfen   1989-12-01         33   

   COMPANYID  
0      15777  
1        946  
2         12  
3        946  
4        946  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 545407 entries, 0 to 545406
Data columns (total 5 columns):
ID              545407 non-null int64
NAME            545407 non-null object
ONMARKETDATE    545407 non-null datetime64[ns]
PRODUCTID       545407 non-null int64
COMPANYID       545407 non-null int64
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 25.0+ MB
None


/Users/markushemmer/PycharmProjects/firstdjango/myvenv/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [112]:
prod_comp_mrg = pd.merge(prod_comp_mrg, product_interaction_class, left_on='PRODUCTID', right_on='PRODUCTID', how='left')
print(prod_comp_mrg.head())
print(prod_comp_mrg.info())

   ID                              NAME ONMARKETDATE  PRODUCTID  COMPANYID  \
0  14  Trental® 400 mg, Retardtabletten   1977-09-01         14      15777   
1  14  Trental® 400 mg, Retardtabletten   1977-09-01         14      15777   
2  21   BETOPTIMA® 5 mg/ml Augentropfen   1985-09-01         21        946   
3  21   BETOPTIMA® 5 mg/ml Augentropfen   1985-09-01         21        946   
4  21   BETOPTIMA® 5 mg/ml Augentropfen   1985-09-01         21        946   

   INTERACTIONCLASSID  
0               166.0  
1           1001088.0  
2                53.0  
3           1000737.0  
4           1000747.0  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 656309 entries, 0 to 656308
Data columns (total 6 columns):
ID                    656309 non-null int64
NAME                  656309 non-null object
ONMARKETDATE          656309 non-null datetime64[ns]
PRODUCTID             656309 non-null int64
COMPANYID             656309 non-null int64
INTERACTIONCLASSID    143834 non-null float64
d

In [ ]:
prod_comp_mrg = prod_comp_mrg.drop('PRODUCTID', 1)

In [ ]:
print(prod_comp_merge.head())

In [ ]:
prod_comp_merge2 = pd.merge(prod_comp_merge, company[['ID','NAME']], left_on="COMPANYID", right_on="ID")
prod_comp_merge2 = prod_comp_merge2[['ID_x','NAME_x','ONMARKETDATE','NAME_y']]
prod_comp_merge2.columns = ['ID','Arzneimittel','Markteintrittsdatum','Hersteller']
prod_comp_merge2 = prod_comp_merge2.sort_values(by='ID', ascending=True)
print(prod_comp_merge2.head())

In [ ]:
product = prod_comp_merge2

In [13]:
# Merge aus Company und Companyadress
comp_adrs_mrg = pd.merge(company, company_adress, left_on='ID', right_on='COMPANYID')
# Straße und Hausnummer zusammenführen
comp_adrs_mrg['Straße und Hausnummer'] = comp_adrs_mrg['STREET'] +' '+ comp_adrs_mrg['STREETNUMBER']
# Unbrauchbare Spalten entfernen
comp_adrs_mrg = comp_adrs_mrg[['ID_x','NAME','ZIP','CITY','Straße und Hausnummer']]
comp_adrs_mrg.columns = ['ID','Hersteller','Postleitzahl','Stadt','Straße und Hausnummer'] # Umbenennung der Spalten
comp_adrs_mrg = comp_adrs_mrg.sort_values(by='ID', ascending=True) # Aufsteigend sortieren
company = comp_adrs_mrg # Finale Zuordnung
print(comp_adrs_mrg.head())

   ID                                        Hersteller Postleitzahl  \
0   1                  AbbVie Deutschland GmbH & Co. KG        65189   
1   3  3M Medica Zweigniederlassung 3M Deutschland GmbH        41453   
2   5                         aar Pharma GmbH u. Co. KG        42853   
3  10                         AktivPharma GmbH & Co. KG        69412   
4  12                                 Alcon Pharma GmbH        79108   

       Stadt Straße und Hausnummer  
0  Wiesbaden       Mainzer Str. 81  
1      Neuss       Hammfelddamm 11  
2  Remscheid          Alleestr. 11  
3   Eberbach         Klausenweg 12  
4   Freiburg      Blankreutestr. 1  


In [ ]:
# Fertigstellen der finalen csv-Dateien

product.to_csv('../data/final/product_final.csv', sep=';', quoting=csv.QUOTE_NONNUMERIC, index=False)
company.to_csv('../data/final/company_final.csv', sep=';', quoting=csv.QUOTE_NONNUMERIC, index=False)
#company_adress.to_csv('../data/final/company_adress_final.csv', sep=';', quoting=csv.QUOTE_NONNUMERIC, index=False)
#product_company.to_csv('../data/final/product_company_final.csv', sep=';', quoting=csv.QUOTE_NONNUMERIC, index=False)
interaction.to_csv('../data/final/interaction_final.csv', sep=';', quoting=csv.QUOTE_NONNUMERIC, index=False)
interaction_class.to_csv('../data/final/interaction_class_final.csv', sep=';', quoting=csv.QUOTE_NONNUMERIC, index=False)
product_interaction_class.to_csv('../data/final/product_interaction_class_final.csv', sep=';', quoting=csv.QUOTE_NONNUMERIC, index=False)
